# Introduction to multislice simulations with *ab*TEM - supplementary

This tutorial is a supplement of a short introduction to image simulation with *ab*TEM covering slightly more advanced topics. The tutorial covers 4D-STEM, frozen phonons and PRISM.

### Contents:

1. <a href='#4d_stem'> 4D-STEM
2. <a href='#frozen_phonons'> The frozen phonon model
3. <a href='#prism'> Large STEM simulation with PRISM
4. <a href='#parallel'> Parallelization

### Author:
* 13/05/2024 Jacob Madsen - For the HyperSpy workshop at ePSIC 2024

In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np

import ase
import abtem

abtem.config.set({"device": "cpu"})
abtem.config.set({"fft": "fftw"})

We recreate some the simulation objects from the basics tutorial. 

In [ ]:
atoms = ase.io.read("sto_lto.cif")

abtem.show_atoms(atoms)

In [ ]:
atoms = ase.io.read("sto_lto.cif")

potential = abtem.Potential(atoms, sampling=0.05, slice_thickness=2)

probe = abtem.Probe(energy=150e3, defocus=50, semiangle_cutoff=20)

scan = abtem.GridScan(
    start=(0, 0),
    end=(1 / 3, 1),
    sampling=probe.ctf.nyquist_sampling,
    fractional=True,
    potential=potential,
)

## 4D-STEM <a id='4d_stem'></a>

To run a 4D-STEM simulation, we only need to change the `AnnularDetector` to a `PixelatedDetector`. The `PixelatedDetector` will by default detect the diffraction patterns up the angle corrsponding to the largest rectangle inside the antialiasing limit, however, we can choose another maximum angle by setting the `max_angle` argument.

Here we set the maximum angle to $100 \ \mathrm{mrad}$.

In [ ]:
detector = abtem.PixelatedDetector(max_angle=100)

We build a scanned multislice simulation, the result is a 4D array with $15\times 43$ probe positions each producing a $116\times 107$ diffraction pattern.

In [ ]:
measurement_4d = probe.scan(scan=scan, potential=potential, detectors=detector)

measurement_4d.array

The size of the dataset is quite manageable in this case, however, 4D-STEM dataset can grow quite large, hence, it is often preferable to write the simulation results directly to disk instead of the memory. *ab*TEM uses Dask and [zarr](https://zarr.readthedocs.io/en/stable/) to efficiently read and write in manageable chunks, in this case 8 chunks across the scan dimensions.

In [ ]:
measurement_4d.to_zarr("measurement_4d.zarr")

We use the `from_zarr` function to lazily read back the results. 

In [ ]:
measurement_4d = abtem.from_zarr("measurement_4d.zarr")
measurement_4d.array

To read the entire measurement from disk into memory, we can run `.compute`, however, this is often unecessary as most of the *ab*TEM features works with lazy measurements. 

We show the diffraction patterns using an interactive visualization reading each chunk directly from disk.

In [ ]:
measurement_4d.show(interact=True)

We can of course also index the dataset to retrieve a specific diffraction pattern.

In [ ]:
measurement_4d[1, 1].show();

Since we have the full diffraction pattern, we can integration between any to scattering angles within the maximum detected angle.

To do this we first need to bin the diffraction patterns radially. We specify 100 radial bins and 1 azimuthal bin.

In [ ]:
polar_binned = measurement_4d.polar_binning(nbins_radial=100, nbins_azimuthal=1)

polar_binned.array

The `to_image_ensemble` creates a representation of the polar binned diffraction patterns for displaying interactively. We interpolate and tile to get a better visualization.

In [ ]:
binned_images = polar_binned.to_image_ensemble().compute().interpolate(0.1).tile((3, 1))

binned_images.show(interact=True)

In 4D-STEM, some algorithm is typically used for reducing the dataset to 2D. *ab*TEM includes some basic algorithms for reduction of 4D-STEM, for more advanced algorithms you may want to try a package dedicated to 4D-STEM.

The center of mass, $\vec{I}_{com}(\vec{r}_p)$, of the diffraction pattern at a probe position, $\vec{r}_p$, may be calculated as

$$
    \vec{I}_{com}(\vec{r}_p) = \int \hat{I}(\vec{k}, \vec{r}_p) \vec{k} d^2\vec{k} \quad ,
$$

where $\hat{I}(\vec{k})$ is a diffraction pattern intensity. Doing this for every diffraction pattern, we obtain the image shown below. The center of mass is returned as complex `Images`, where the real and imaginary parts correspond to the $x$- and $y$-direction, respectively. We set `units="reciprocal"`, hence each complex component is in units of $\mathrm{Å}^{-1}$.

In [ ]:
center_of_mass = measurement_4d.center_of_mass(units="1/Å")
center_of_mass.array

We interpolate and tile before showing the real part of the COM.

In [ ]:
interpolated_center_of_mass = center_of_mass.interpolate(0.1).tile((3, 1))

interpolated_center_of_mass.real().show(title="COM real part", cbar=True);

We can show both real and imaginary components using [domain coloring](https://en.wikipedia.org/wiki/Domain_coloring).

In [ ]:
interpolated_center_of_mass.show(cbar=True, cmap="hsluv", title="COM domain coloring");

It may be shown, in the weak-phase approximation, that by integrating $\vec{I}_{com}(\vec{r}_p)$, we can obtain the phase change of the exit wave, $\phi(\vec{r_p})$, cross-correlated with the probe intensity

$$
\vec{I}_{iCOM}(\vec{r}_p) = \frac{1}{2\pi} \left[\|\psi_0(\vec{r})\|^2 \star \phi(\vec{r})\right](\vec{r}_p) \quad .
$$

This is the so-called integrated center of mass. We can calculate this using the `integrate_gradient` method, which assumes a complex `Image`.

In [ ]:
integrated_gradient = interpolated_center_of_mass.integrate_gradient()

integrated_gradient.show();

## The frozen phonon model <a id='frozen_phonons'></a>
The atoms in any real material at a particular instance of time are not exactly located at their symmetrical lattice points due to thermal vibrations. In the Frozen phonon approximation, the effect of thermal vibrations are simulated by the _intensities_ averaged over several different configurations of atoms with different random offsets. 

To simulate frozen phonons the `Atoms` are wrapped with a `FrozenPhonons` object, where we also need to provide the magnitude of the thermal vibrations for each atomic species and the number of configurations we average over. Including more configurations will be more accurate, but of course also more expensive to calculate. 

Getting the right magnitude of thermal vibrations for a particular material, is not always trivial, here we just use the same reasonable value of $0.1 \ \mathrm{Å}$ for all the atomic numbers. We set the number og random structures in the thermal ensemble to 8.

In [ ]:
atoms

In [ ]:
sigmas = {"O": .1, "Sr": .1, "La":.1, "Ti":.1}

frozen_phonons = abtem.FrozenPhonons(atoms * (1, 1, 4), sigmas=sigmas, num_configs=8)

We can draw a particular frozen phonon configuration by iterating.

In [ ]:
iterator = iter(frozen_phonons)

In [ ]:
config = next(iterator)

abtem.show_atoms(config, scale=.5);

The potential can be created as above, we just provide the frozen phonons instead of the atoms. 

In [ ]:
frozen_phonon_potential = abtem.Potential(
    frozen_phonons, sampling=0.05, slice_thickness=2
)

In [ ]:
frozen_phonon_potential.build().project().compute().show(interact=True)

The potential object can be used in the same way as above, here we do a CBED simulation for each thermal snapshot. The result is an ensemble of 8 wave functions.

In [ ]:
exit_waves = probe.multislice(potential=frozen_phonon_potential)

exit_waves.array

In [ ]:
exit_waves.compute()

We show the ensemble of wave functions interactively.

In [ ]:
exit_waves.diffraction_patterns().shape

In [ ]:
exit_waves.diffraction_patterns().show(interact=True)

To get final diffraction pattern, we take the mean over the ensemble dimension.

__Note__: some imaging modes will average over frozen phonons by default to conserve memory.

In [ ]:
probe.cutoff_angles

In [ ]:
mean_diffraction_pattern = exit_waves.diffraction_patterns().mean(0)

In [ ]:
#exit_waves.diffraction_patterns().

We show the resulting diffraction pattern on a power scale.

In [ ]:
mean_diffraction_pattern.show(power=0.25, units="mrad");

## Large STEM simulation with PRISM <a id='prism'></a>
Multslice simulations of STEM images can be very slow because the scattering of the electron probe is calculated from scratch at each pixel of the image. An alternative is to use the [PRISM algorithm](https://prism-em.com). 

The PRISM algorithm for STEM simulations may be summarised as two consecutive stages: 

__Multislice stage:__ Build the PRISM scattering matrix by applying the multislice algorithm to each component of a plane wave expansion of the incoming probe.

__Reduction stage:__ For each probe position, perform a reduction with a set of expansion coefficients determined by the probe position and phase aberrations.

PRISM almost always provides a decent speed-up for images with many probe positions, and allows for huge speedups, at a modest cost to accuracy, by using Fourier interpolation.

We import the moderately large model of a nanoparticle on carbon that was constructed in "atomic_models_with_ase.ipynb".

In [ ]:
from ase.io import read

cluster = read("cluster_on_carbon.cfg")

print(
    "Number of atoms: {} \nCell: {:.2f} x {:.2f} x {:.2f}".format(
        len(cluster), *np.diag(cluster.cell)
    )
)

In [ ]:
abtem.show_atoms(cluster, plane="xz", title="Side view")

In *ab*TEM, the PRISM algorithm can be used by exchanging the `Probe` for the `SMatrix`. However, we need to provide the potential directly to the `SMatrix`, additionally there is new to know about, namely interpolation.

Increasing the interpolation decreases the number of plane waves necessary in the plane wave expansion of the probe, hence lowering both time and memory consumption. The price is that the interpolation factor also decreases the Fourier space sampling rate, i.e. the pixels of the detected diffraction patterns becomes larger and equivalently the probe window becomes smaller.

In [ ]:
cluster_potential = abtem.Potential(cluster, gpts=768, slice_thickness=2)

S = abtem.SMatrix(
    potential=cluster_potential, interpolation=6, energy=150e3, semiangle_cutoff=25
)

S.cutoff_angles

We can get the equivalent probe produced represented by the scattering matrix using the `dummy_probes` method.

In [ ]:
S.dummy_probes(plane="exit").show()

The nanoparticle is located around $60 \ \mathrm{Å}$ inside the cell, hence we may want to change the focus to match. Remember the convention used in *ab*TEM; positive defocus negates positive free-space propagation.

In [ ]:
ctf = abtem.CTF(defocus=[0, 70])

S.dummy_probes(plane="exit", ctf=ctf).show(interact=True)

We can `build` the `SMatrix` to produce an `SMatrixArray`. The `SMatrixArray` wraps a 3D NumPy array where the first dimension indexes the plane wave of the plane wave expansion.

We see that the expansion of our probe requires 177 plane waves.

In [ ]:
S.build().array

We can access the plane waves using the `waves` property.

In [ ]:
plane_waves = S.build().waves.compute()

We show every 10'th of the plane waves using domain coloring.

In [ ]:
plane_waves.shape

In [ ]:
plane_waves[::50].to_images().show(
    cbar=True, cmap="hsv", explode=True, common_color_scale=True
)

We create a MAADF detector.

In [ ]:
maadf_detector = abtem.AnnularDetector(inner=60, outer=150)

We can run the simulation by just providing the detector. By not providing a scan object we assume scanning the Nyquist sampling rate across the entire sample.

In [ ]:
image_prism = S.scan(
    detectors=maadf_detector,
    max_batch_reduction=50,
    ctf=ctf,
)

We see that the result will be 2 images of $170 \times 170$, i.e. $2 \times 28900$ probe positions. This should be compared to the only $177$ plane waves in the expansion, we thus reduced the number of runs of the multislice algorithm by a factor of $326$. The caveat is that we still need to run the reduction stage twice and we lost some reciprocal space resolution by using interpolation.

In [ ]:
image_prism.array

We compute to run both the PRISM multislice and reduction stages. The simulation took around 40 seconds on my 2018 MacBook.

In [ ]:
image_prism.compute()

Lastly, we apply the typical postprocessing steps and show the MAADF image.

In [ ]:
image_prism.gaussian_filter(0.35).interpolate(0.1).show(interact=True);

## Parallelization and memory <a href='#parallel'>
   
*ab*TEM is parallelized using [Dask](https://www.dask.org/){cite}`dask`, which is a flexible library for parallel computing in Python. 

When running methods such as `build`, `multislice` and `scan`, *ab*TEM creates a Dask [*task graph*](https://docs.dask.org/en/stable/graphs.html). The task graph breaks down a larger task into smaller subtasks represented by the *node*, with *edges* between nodes if it the subtask dependent on another subtask. 
    
After generating a task graph, it needs to be executed on (parallel) hardware. This is the job of a [task scheduler](https://docs.dask.org/en/stable/scheduler-overview.html). Dask provides several *task schedulers*: each of which will compute a task graph and give the same result, but with different performance characteristics. The default scheduler is the [`ThreadPoolExecutor`](https://docs.dask.org/en/stable/scheduling.html#local-threads). The threaded scheduler takes the argument `num_workers`, which sets the number of threads to use.

As an example, we create the task graph for the frozen phonon simulation above. We set the scheduler explicitly and change the number of workers to 2. Note that changing the number of workers may not always affect the computational time significantly, as NumPy and the FFT libraries also has parallelism.

In [ ]:
exit_waves = probe.multislice(potential=frozen_phonon_potential)

exit_waves.array

In [ ]:
exit_waves = probe.multislice(potential=frozen_phonon_potential)

exit_waves.compute(scheduler="threads", num_workers=2)

The Dask distributed scheduler is necessary for running your simulations on a cluster, but it also runs [locally on a personal machine](https://docs.dask.org/en/stable/scheduling.html#dask-distributed-local). 

You can use the Dask distributed scheduler by initializing a [Dask Client](https://distributed.dask.org/en/stable/client.html). The `Client` takes keyword arguments such as `n_workers` (note that this is different from `num_workers` above!) and `threads_per_worker`.

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4, threads_per_worker=1)

client

A benefit of using the distributed scheduler on a single machine is the live diagnostic dashboard. You can access this through the Dashboard link shown above.

We run the frozen phonons calculation with the dsitributed scheduler as an example. When a client is active we should not provide any arguments to `compute`.

In [ ]:
exit_waves = probe.multislice(potential=frozen_phonon_potential)

exit_waves.compute()